In [3]:
# we scraper
from bs4 import BeautifulSoup

# request
import requests

# io
from io import BytesIO

# date parser and sleep
from dateutil.parser import parse
import time

# web driver
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

# reeading and exporting to excel files
import pandas as pd


# looping process monitoring
from tqdm import tqdm
import random

In [4]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

s = slice(11)

In [5]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("SEALAND")

In [6]:
len(bl_list)

39

In [11]:
bl_list

['227145963',
 '227927058',
 '228000484',
 '228698779',
 '228667042',
 '228487087',
 '228478181',
 '228495628',
 '228528205',
 '228576712',
 '228151079',
 '228326741',
 '228329054',
 '228776329',
 '228627103',
 '228107228',
 '228404249',
 '228422054',
 '228876620',
 '228883601',
 '228877614',
 '229154632',
 '229327088',
 '232450086',
 '232588354',
 '232587161',
 '232643988',
 '232657462',
 '232841994',
 '231993522',
 '231993496',
 '232105329',
 '232134042',
 '232248681',
 '232306683',
 '232293144',
 '232333432',
 '232333360',
 '232938268']

In [7]:
dict_hasil = {}
current_dict = {}
list_dict = []
gagal = []

In [8]:
bl_list = ["232938268"]

**FIX**


In [31]:
list_dict = []
gagal = []
# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
driver.get('https://www.maersk.com/tracking/' + bl_list[random.randint(0, len(bl_list)-1)])

# wait to load all
time.sleep(2)

#click allow all
click_allowall = driver.find_element(By.XPATH,'//button[text()="Allow all"]')
click_allowall.click()

for i, bls in enumerate(tqdm(bl_list)):
    try:
        # access the web
        driver.get('https://www.maersk.com/tracking/' + bl_list[i])
        # sleep wait
        title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='maersk-app']/div/div/div/dl")))
        time.sleep(3)
        
        # finding how many hide details in 1 page refering to how many container number in 1 bl
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        containers = soup1.find_all('div', {"data-test": "container"})
        number_of_ctr = len(containers)
        
        print(bls, "HAVE ", number_of_ctr, " CTR")
        # print(dict_hasil)

        if number_of_ctr > 1:
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                #
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            print("Hasil dari Append Judul")
            print(dict_hasil)
            if driver.find_element(By.CLASS_NAME, "track__feedback"):
                x=4
            else:
                x=3
            # clicking all view more
            for c in range(number_of_ctr):
                try:
                    print("clicking div no :",  x)
                    show_details2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/div/div/div[{}]/dl/button'.format(x))))  
                    # /html/body/main/div/div/div/div/div[3]/dl/button/span
                    show_details_check = driver.find_element(By.XPATH, "/html/body/main/div/div/div/div/div[{}]/dl/button/span".format(x))
                    if show_details_check.text == 'Show details':
                        show_details2.click()
                    print(show_details_check.text)
                    
                    time.sleep(1)
                except Exception as e:
                    print(e)
                x=x+1

            time.sleep(2)
            # taking the whole page's html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)
                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
                 
                for panjang in range(0, len(milestones), 2):
                    key = milestones[panjang]
                    value = milestones[panjang + 1]

                    if key == 'Container Number':
                        if current_dict:
                            list_dict.append(current_dict.copy())
                        current_dict = {'Container Number': value}
                    else:
                        current_dict[key] = value

                    current_dict.update(dict_hasil) 
                print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
            print(bls, ", completed!!")
        else:
            time.sleep(1)
            
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)

                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
            
            for t, milestone in enumerate(milestones):
                if milestone == "Container Number":
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                elif t%2==0 :
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                    
            current_dict.update(dict_hasil) 
            print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
        
        unique_list = []
        for d in list_dict:
            if d not in unique_list:
                unique_list.append(d)
                
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/1 [00:00<?, ?it/s]

232938268 HAVE  5  CTR
['Bill of Lading number', '232938268', 'From', 'Jakarta', 'To', 'Haiphong']
Hasil dari Append Judul
{'Bill of Lading number': '232938268', 'From': 'Jakarta', 'To': 'Haiphong'}
clicking div no : 4
Hide details
clicking div no : 5
Hide details
clicking div no : 6
Hide details
clicking div no : 7
Hide details
clicking div no : 8
Message: 
Stacktrace:
	GetHandleVerifier [0x00FA72A3+45731]
	(No symbol) [0x00F32D51]
	(No symbol) [0x00E2880D]
	(No symbol) [0x00E5B940]
	(No symbol) [0x00E5BE0B]
	(No symbol) [0x00E8D1F2]
	(No symbol) [0x00E78024]
	(No symbol) [0x00E8B7A2]
	(No symbol) [0x00E77DD6]
	(No symbol) [0x00E531F6]
	(No symbol) [0x00E5439D]
	GetHandleVerifier [0x012B0716+3229462]
	GetHandleVerifier [0x012F84C8+3523784]
	GetHandleVerifier [0x012F214C+3498316]
	GetHandleVerifier [0x01031680+611968]
	(No symbol) [0x00F3CCCC]
	(No symbol) [0x00F38DF8]
	(No symbol) [0x00F38F1D]
	(No symbol) [0x00F2B2C7]
	BaseThreadInitThunk [0x777CFCC9+25]
	RtlGetAppContainerNamedObjec

100%|██████████| 1/1 [00:24<00:00, 24.39s/it]

{'Container Number': 'TEMU7408643', 'Bill of Lading number': '232938268', 'From': 'Jakarta', 'To': 'Haiphong', 'Jakarta Gate outEmpty': '04 Nov 2023', 'Jakarta Gate in': '05 Nov 2023', 'Jakarta Load': '08 Nov 2023', 'Jakarta Vessel departure': '08 Nov 2023', 'Haiphong Vessel arrival': '15 Nov 2023', 'Haiphong Discharge': '15 Nov 2023', 'Haiphong Gate out for delivery': '18 Nov 2023', 'Haiphong Empty container return': '19 Nov 2023'}
{'Container Number': 'TCNU2539569', 'Bill of Lading number': '232938268', 'From': 'Jakarta', 'To': 'Haiphong', 'Jakarta Gate outEmpty': '02 Nov 2023', 'Jakarta Gate in': '07 Nov 2023', 'Jakarta Load': '08 Nov 2023', 'Jakarta Vessel departure': '08 Nov 2023', 'Haiphong Vessel arrival': '15 Nov 2023', 'Haiphong Discharge': '15 Nov 2023', 'Haiphong Gate out for delivery': '16 Nov 2023', 'Haiphong Empty container return': '17 Nov 2023'}
{'Container Number': 'HASU4470647', 'Bill of Lading number': '232938268', 'From': 'Jakarta', 'To': 'Haiphong', 'Jakarta Gate o

In [32]:
unique_list

[{'Container Number': 'TEMU7408643',
  'Bill of Lading number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Jakarta Gate outEmpty': '04 Nov 2023',
  'Jakarta Gate in': '05 Nov 2023',
  'Jakarta Load': '08 Nov 2023',
  'Jakarta Vessel departure': '08 Nov 2023',
  'Haiphong Vessel arrival': '15 Nov 2023',
  'Haiphong Discharge': '15 Nov 2023',
  'Haiphong Gate out for delivery': '18 Nov 2023',
  'Haiphong Empty container return': '19 Nov 2023'},
 {'Container Number': 'TCNU2539569',
  'Bill of Lading number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Jakarta Gate outEmpty': '02 Nov 2023',
  'Jakarta Gate in': '07 Nov 2023',
  'Jakarta Load': '08 Nov 2023',
  'Jakarta Vessel departure': '08 Nov 2023',
  'Haiphong Vessel arrival': '15 Nov 2023',
  'Haiphong Discharge': '15 Nov 2023',
  'Haiphong Gate out for delivery': '16 Nov 2023',
  'Haiphong Empty container return': '17 Nov 2023'},
 {'Container Number': 'HASU4470647',
  'Bill of Lading number': '232938268',
 

In [33]:
filtered_dict_list =[]
for dict_milestone in unique_list:
    # print(dict_milestone)
    kw = ['Bill of Lading number', 'Container Number', 'From', 'To', dict_milestone['From'], dict_milestone['To']]
    dict_key = []
    for keys in dict_milestone:
        dict_key.append(keys)
    # print("BEFORE", dict_key)
    new_list = [] 
    
    for key in dict_key:
        if any(substring in key for substring in kw):
            new_list.append(key)
    # print("AFTER",new_list)
    dict_milestone = {key: dict_milestone[key] for key in new_list}
    filtered_dict_list.append(dict_milestone)

In [34]:
filtered_dict_list

[{'Container Number': 'TEMU7408643',
  'Bill of Lading number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Jakarta Gate outEmpty': '04 Nov 2023',
  'Jakarta Gate in': '05 Nov 2023',
  'Jakarta Load': '08 Nov 2023',
  'Jakarta Vessel departure': '08 Nov 2023',
  'Haiphong Vessel arrival': '15 Nov 2023',
  'Haiphong Discharge': '15 Nov 2023',
  'Haiphong Gate out for delivery': '18 Nov 2023',
  'Haiphong Empty container return': '19 Nov 2023'},
 {'Container Number': 'TCNU2539569',
  'Bill of Lading number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Jakarta Gate outEmpty': '02 Nov 2023',
  'Jakarta Gate in': '07 Nov 2023',
  'Jakarta Load': '08 Nov 2023',
  'Jakarta Vessel departure': '08 Nov 2023',
  'Haiphong Vessel arrival': '15 Nov 2023',
  'Haiphong Discharge': '15 Nov 2023',
  'Haiphong Gate out for delivery': '16 Nov 2023',
  'Haiphong Empty container return': '17 Nov 2023'},
 {'Container Number': 'HASU4470647',
  'Bill of Lading number': '232938268',
 

In [35]:
list_of_dict_fix = []
for filter_dict in filtered_dict_list:
    replacement_mapping = {
        filter_dict["From"]: 'Origin',
        filter_dict["To"]: 'Destination'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    list_of_dict_fix.append(updated_dict)

In [37]:
list_of_dict_fix

[{'Container Number': 'TEMU7408643',
  'Bill of Lading number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Origin Gate outEmpty': '04 Nov 2023',
  'Origin Gate in': '05 Nov 2023',
  'Origin Load': '08 Nov 2023',
  'Origin Vessel departure': '08 Nov 2023',
  'Destination Vessel arrival': '15 Nov 2023',
  'Destination Discharge': '15 Nov 2023',
  'Destination Gate out for delivery': '18 Nov 2023',
  'Destination Empty container return': '19 Nov 2023'},
 {'Container Number': 'TCNU2539569',
  'Bill of Lading number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Origin Gate outEmpty': '02 Nov 2023',
  'Origin Gate in': '07 Nov 2023',
  'Origin Load': '08 Nov 2023',
  'Origin Vessel departure': '08 Nov 2023',
  'Destination Vessel arrival': '15 Nov 2023',
  'Destination Discharge': '15 Nov 2023',
  'Destination Gate out for delivery': '16 Nov 2023',
  'Destination Empty container return': '17 Nov 2023'},
 {'Container Number': 'HASU4470647',
  'Bill of Lading number'

In [42]:
list_of_dict_fix2 = []

for filter_dict in list_of_dict_fix:
    replacement_mapping = {
        "Bill of Lading number": "BL Number",
        "Origin Vessel departure": 'ATD',
        "Destination Discharge": 'ATA',
        "Destination Gate out": 'Container Release',
        "Destination Empty container return": 'Container Return',
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            if old_key in key:
                key = new_key

        updated_dict[key] = value

        if is_date(value):
            input_date = datetime.strptime(value, '%d %b %Y')
            updated_dict[key] = input_date.strftime("%Y-%m-%d")

    updated_dict["Liners"] = "SEALAND"

    list_of_dict_fix2.append(updated_dict)

In [43]:
list_of_dict_fix2

[{'Container Number': 'TEMU7408643',
  'BL Number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Origin Gate outEmpty': '2023-11-04',
  'Origin Gate in': '2023-11-05',
  'Origin Load': '2023-11-08',
  'ATD': '2023-11-08',
  'Destination Vessel arrival': '2023-11-15',
  'ATA': '2023-11-15',
  'Container Release': '2023-11-18',
  'Container Return': '2023-11-19',
  'Liners': 'SEALAND'},
 {'Container Number': 'TCNU2539569',
  'BL Number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Origin Gate outEmpty': '2023-11-02',
  'Origin Gate in': '2023-11-07',
  'Origin Load': '2023-11-08',
  'ATD': '2023-11-08',
  'Destination Vessel arrival': '2023-11-15',
  'ATA': '2023-11-15',
  'Container Release': '2023-11-16',
  'Container Return': '2023-11-17',
  'Liners': 'SEALAND'},
 {'Container Number': 'HASU4470647',
  'BL Number': '232938268',
  'From': 'Jakarta',
  'To': 'Haiphong',
  'Origin Gate outEmpty': '2023-11-03',
  'Origin Gate in': '2023-11-06',
  'Origin Load': '20

In [27]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict_fix2)

ConfigurationError: The resolution lifetime expired after 20.217 seconds: Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:10.1.200.99@53 answered The DNS operation timed out.; Server Do53:10.1.200.107@53 answered The DNS operation timed out.; Server Do53:192.168.34.4@53 answered The DNS operation timed out.; Server Do53:192.168.100.17@53 answered The DNS operation timed out.; Server Do53:192.168.1.1@53 answered The DNS operation timed out.; Server Do53:10.1.200.99@53 answered The DNS operation timed out.; Server Do53:10.1.200.107@53 answered The DNS operation timed out.; Server Do53:192.168.34.4@53 answered The DNS operation timed out.; Server Do53:192.168.100.17@53 answered The DNS operation timed out.

In [23]:
gagal

['228883601',
 '228877614',
 '229154632',
 '229327088',
 '232450086',
 '232588354',
 '232587161',
 '232643988',
 '232657462',
 '232841994',
 '231993522',
 '231993496',
 '232105329',
 '232134042',
 '232248681',
 '232306683',
 '232293144',
 '232333432',
 '232333360',
 '232938268']

In [ ]:
['232546209',
 '232643988',
 '231993708',
 '231993522',
 '231993496',
 '232105329',
 '232134042',
 '232306683',
 '232333432']

In [63]:
df = pd.DataFrame.from_dict(list_of_dict_fix2)
df.to_excel('sealand_maintained.xlsx')